ある特定のRIC（インデックス）を取り上げ、東証３３業種の指数との相関を見ます。<BR>
相関係数の計算期間やデータの頻度などを設定して相関の高い業種を探し出します。<BR>
さらに、最も正相関の高い業種との相関係数の推移をヒストリカルに見てみます。

In [ ]:
#ライブラリーインポート、APPキー入力

import eikon as ek
ek.set_app_key('DEFAULT_CODE_BOOK_APP_KEY')

import pandas as pd #data analysis and manipulation tool
import numpy as np
import matplotlib.pyplot as plt

import datetime as dt #date and time module
from datetime import date 
from datetime import datetime
import dateutil.relativedelta
from dateutil.relativedelta import relativedelta, FR

import ipywidgets as widgets #Widget
from ipywidgets import Box

import plotly
import plotly.graph_objects as go
import plotly.offline as offline
from plotly.subplots import make_subplots

import warnings
warnings.filterwarnings('ignore')




In [ ]:
#ウィジェット設定

Index = widgets.Text(
    value = 'US10YT=RR',
    description = 'RIC',
    disabled = False,) #比べたいインデックス取得

End = widgets.DatePicker(
    value = datetime(2021,10,8).date(),
    description='End Date',
    disabled=False,) #いつまでの期間で相関計算するか？


T = widgets.Dropdown(
    options = ['0.0833333','0.25','0.5','1','2','3','4','5','7','10'],
    value = '1',
    description = '期間（年）',
    disabled = False,) #相関計算期間の長さ設定

Intv = widgets.Dropdown(
    options = ['daily','weekly','monthly'],
    value = 'daily',
    description = 'データ頻度',
    style = {'description_width': 'initial'},
    disabled = False,) #データの頻度設定（日次、週次、月次）


items = [Index, End, T, Intv]
box = Box(children = items)
box

In [ ]:
#ヒストリカルデータ取得するRICのリスト作成

INDEX_T33 = [Index.value,'.TOPX','.IGLSS.T','.IRUBR.T','.ISVCS.T','.IFINS.T','.IMISC.T','.IPAPR.T','.IPHAM.T','.IWHOL.T','.ICHEM.T',
             '.ISHIP.T','.IMCHN.T','.IMETL.T','.IBNKS.T','.IAIRL.T','.ICNST.T','.IMING.T','.IRETL.T','.ISECU.T',
             '.ICOMS.T','.IFOOD.T','.IFISH.T','.IPRCS.T','.IPETE.T','.ITXTL.T','.IWHSE.T','.ISTEL.T','.IEPNG.T',
              '.IELEC.T','.INFRO.T','.IRLTY.T','.IINSU.T','.ITEQP.T','.IRAIL.T']
INDEX_T33 #東証３３業種とトピックスに比べたいインデックスを加えたリスト確認

In [ ]:
#ヒストリカルデータ取得範囲の設定

s = (End.value + relativedelta(months=int((float(T.value)+0.000001)*-12))) #計算期間の起点
e = End.value #計算期間の終点
print(s,type(s)) #起点日付とそのデータ属性確認
print(e,type(e)) #終点日付とそのデータ属性確認

In [ ]:
#ヒストリカルデータ取得の関数定義

def historical_data(ric,startdate, enddate):
    
        
    hdata = ek.get_timeseries(ric,
                              start_date = f'{startdate}',
                              end_date = f'{enddate}',
                              fields = ['TIMESTAMP','CLOSE'],
                              interval = Intv.value)
    return hdata


In [ ]:
#ヒストリカルデータ取得とデータフレーム作成


data = pd.DataFrame() #dataというデータフレーム設定

for i in INDEX_T33: #dfにインデックスおよび各業種の引け値データを取り込む
    df = historical_data(i,s,e)
    df.rename(columns = {'CLOSE': i}, inplace = True)    # inplace = True … 元のデータに反映させる
    if len(data):
        data = pd.concat([data, df], axis=1) #インデックスと各業種のデータを連結させる
    else:
        data = df

data = data.fillna(method="ffill") #データ欠損は前データで代用する 
data


In [ ]:
#相関係数の計算（相関行列）

Corr = data.corr() #Corrを相関行列に

#Indexとの相関係数に関連する部分だけ抽出

Index_corr = Corr.iloc[:,0] #インデックスとの相関部分だけ抽出
ABS=Index_corr.abs().sort_values(ascending=False) #絶対値に変換し降順にソート

Ichiran=pd.concat([ABS,Index_corr],axis=1) #相関係数：左　とその絶対値：右　を連結し絶対値の降順で表示
Ichiran

In [ ]:
#相関係数（絶対値）最大値の業種とその相関係数の出力

print(Index.value,s,e)
MXgyoushu = ABS[1:].idxmax() #相関係数絶対値最大値の業種をRICで取得
MXname, err = ek.get_data(instruments=MXgyoushu,fields='DSPLY_NMLL') #最大値業種名を日本語で
print('最大値',MXname,Index_corr.loc[MXgyoushu])


In [ ]:
#相関係数（絶対値）最小値の業種とその相関係数の出力

print(Index.value,s,e)
MIgyoushu = ABS.idxmin() #相関係数絶対値最小値の業種をRICで取得
MIname, err = ek.get_data(instruments=MIgyoushu,fields='DSPLY_NMLL') #最小値業種名を日本語で
print('最小値',MIname,Index_corr.loc[MIgyoushu])

In [ ]:
#インデックスと相関１位業種との散布図作成

CHdataX = (data.loc[:,MXgyoushu]) #X軸データ
CHdataY = (data.iloc[:,0]) #Y軸データ

trace = go.Scatter(x=CHdataX,y=CHdataY,mode='markers')
layout = go.Layout(xaxis=dict(title=MXgyoushu),yaxis=dict(title=Index.value),title="インデックスと相関１位業種　散布図")

fig = dict(data = [trace],layout=layout)
offline.iplot(fig)

In [ ]:
#時系列で相関係数を見る業種選択

PICKUP = widgets.Dropdown(
    options = ['相関１位','.TOPX','.IGLSS.T','.IRUBR.T','.ISVCS.T','.IFINS.T','.IMISC.T','.IPAPR.T','.IPHAM.T','.IWHOL.T','.ICHEM.T',
             '.ISHIP.T','.IMCHN.T','.IMETL.T','.IBNKS.T','.IAIRL.T','.ICNST.T','.IMING.T','.IRETL.T','.ISECU.T',
             '.ICOMS.T','.IFOOD.T','.IFISH.T','.IPRCS.T','.IPETE.T','.ITXTL.T','.IWHSE.T','.ISTEL.T','.IEPNG.T',
              '.IELEC.T','.INFRO.T','.IRLTY.T','.IINSU.T','.ITEQP.T','.IRAIL.T'],
    value = '相関１位',
    description = '時系列業種選択',
    disabled = False,) #時系列で相関を見る業種選択

items = [PICKUP]
box = Box(children = items)
box

In [ ]:
#相関係数を時系列で取得

if PICKUP.value=='相関１位':
    GYOUSHU = MXgyoushu
else:
    GYOUSHU = PICKUP.value
    
HISdata = pd.DataFrame() #ヒストリカルマーケットデータ格納データフレーム設定
HIScorrD = pd.DataFrame(columns=['Corr']) #ヒストリカル相関係数格納データフレーム設定

N = np.arange(121) #ヒストリカル期間設定（月数）
for n in N:
    et = End.value
    et = et + relativedelta(months=-1*n)
    st = et + relativedelta(months=int((float(T.value)+0.000001)*-12))
        
    HISdata = historical_data([Index.value,GYOUSHU],st,et) #各時点のヒストリカルデータ
            
    HIScorr = HISdata.corr().iloc[0,1] #各時点のヒストリカル相関係数
    
    HIScorrD.loc[et] = HIScorr #各時点のヒストリカル相関係数を集計
    
HIScorrD

In [ ]:
#相関係数の推移をチャート化

D = HIScorrD.index
X = HIScorrD.iloc[:,0]

trace = go.Scatter(x=D, y=X, mode='lines')
layout = go.Layout(xaxis = dict(title='Date', type='date', dtick='M6'),
                  yaxis = dict(title=GYOUSHU+'の相関係数'),
                  title = '相関係数の推移')

fig = dict(data = [trace], layout = layout)
offline.iplot(fig)

In [ ]:
#インデックスと選択した業種の過去10年ヒストリカルチャート作成

CHdataI = historical_data(Index.value,e+relativedelta(months=-120),e)
CHdataI = CHdataI.rename(columns={'CLOSE':'CHdataI'}) 

CHdataG = historical_data(GYOUSHU,e+relativedelta(months=-120),e)
CHdataG = CHdataG.rename(columns={'CLOSE':'CHdataG'}) 

TOPIX = historical_data('.TOPX',e+relativedelta(months=-120),e)
TOPIX = TOPIX.rename(columns={'CLOSE':'TOPIX'}) 

CHdata=CHdataI.join([TOPIX,CHdataG]).fillna(method="ffill") #連結

trace1 = go.Scatter(x=CHdata.index,y=CHdata.CHdataI,mode='lines',name=Index.value,yaxis='y',line=dict(color="blue"))
trace2 = go.Scatter(x=CHdata.index,y=CHdata.CHdataG,mode='lines',name=GYOUSHU,yaxis='y2',line=dict(color="red")) 
trace3 = go.Scatter(x=CHdata.index,y=CHdata.TOPIX,mode='lines',name='TOPIX',yaxis='y3',line=dict(color="gray")) 

layout = go.Layout(xaxis = dict(title='Date', type='date', dtick='M6', domain=[0.0,0.9]),
                   yaxis = dict(title=Index.value, side='left', color="blue"),
                   yaxis2 = dict(title=GYOUSHU, side='right', overlaying='y', color="red"),
                   yaxis3 = dict(title='TOPIX', side='right', overlaying='y', position=1.0, color="gray"),
                   legend = {"x":0.9,"y":1.3},
                   title = 'マーケットデータ')

fig = dict(data = [trace1,trace2,trace3], layout = layout)
offline.iplot(fig)

